## **Capítulo 5: Análise de Risco**

Neste capítulo, vamos além das métricas simples de retorno e volatilidade. O objetivo é entender o que fazer com os dados de retorno para realizar uma análise de risco mais profunda, comparando diferentes ativos e quantificando perdas potenciais. Vamos ver que, em geral, para obter maiores retornos, devemos estar dispostos a assumir mais riscos.

### **5.1 Análise Visual dos Retornos**

Um ponto fundamental na análise de qualquer ativo financeiro é a observação dos dados. Vamos começar baixando uma série de preços mais longa do IBOVESPA e visualizar a diferença entre a série de **preços** e a série de **retornos**.

* A série de **preços** geralmente apresenta **tendências** (de alta ou de baixa).
* A série de **retornos**, por outro lado, tende a ser **estacionária**, ou seja, oscila em torno de uma média próxima de zero, sem uma tendência clara.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Vamos baixar 10 anos de dados do IBOVESPA
ibov = yf.download('^BVSP', start='2014-01-01', end='2023-12-31', auto_adjust=True)

# --- Gráfico 1: Série de Preços ---
ibov['Close'].plot(figsize=(12, 6), title='Série de Preços do IBOVESPA (com tendência)')
plt.ylabel('Pontos')
plt.grid(True)
plt.show()


# --- Gráfico 2: Série de Retornos ---
# Calculando os retornos logarítmicos
ibov['retorno_log'] = np.log(ibov['Close']).diff()

# Plotando a série de retornos
ibov['retorno_log'].plot(figsize=(12, 6), title='Série de Retornos do IBOVESPA (estacionária, oscila em torno de zero)')
plt.ylabel('Retorno Logarítmico')
plt.grid(True)
plt.show()

### **5.2.1 Análise Estatística: Medidas Descritivas**

Além da análise visual, usamos medidas estatísticas para descrever e quantificar o comportamento da série de retornos. Além de média, mediana e desvio padrão, duas medidas de "forma" são muito importantes:

* **Assimetria (Skewness):** Mede a simetria da distribuição dos retornos.
    * `skew > 0` (Positiva): A cauda direita é mais longa. Indica que houve ocorrências de ganhos extremos mais frequentes que perdas extremas.
    * `skew < 0` (Negativa): A cauda esquerda é mais longa. Indica que perdas extremas foram mais frequentes que ganhos extremos.

* **Curtose (Kurtosis):** Mede o "peso" das caudas da distribuição, ou seja, a probabilidade de ocorrerem eventos extremos (tanto positivos quanto negativos).
    * Uma distribuição Normal tem curtose de 3.
    * `kurtosis > 3`: Chamada de "leptocúrtica" ou "caudas pesadas". Significa que eventos extremos são **mais prováveis** de acontecer do que em uma distribuição Normal. Retornos de ativos financeiros tipicamente apresentam essa característica.

In [ ]:
# Vamos usar a série de retornos do IBOV que já calculamos
retornos_ibov = ibov['retorno_log'].dropna()

# Medidas já conhecidas
media = retornos_ibov.mean()
mediana = retornos_ibov.median()
desvio_padrao = retornos_ibov.std()
minimo = retornos_ibov.min()
maximo = retornos_ibov.max()

# Novas medidas: Assimetria e Curtose
assimetria = retornos_ibov.skew()
curtose = retornos_ibov.kurt() + 3 # O Pandas calcula o "excesso" de curtose, então somamos 3 para comparar com a Normal

print("--- Estatísticas Descritivas dos Retornos do IBOV ---")
print(f"Média: {media:.5%}")
print(f"Desvio Padrão (Volatilidade): {desvio_padrao:.5%}")
print(f"Assimetria (Skewness): {assimetria:.4f}")
print(f"Curtose (Kurtosis): {curtose:.4f}")

if curtose > 3:
    print("\nConclusão: A distribuição tem caudas pesadas (leptocúrtica), o que é típico para retornos de ativos financeiros.")

### **5.2.2 Visualizando a Distribuição: Histograma**

Um histograma é a melhor forma de visualizar a distribuição dos retornos que acabamos de descrever com as estatísticas. Ele nos mostra quais faixas de retorno são mais frequentes.

Para enriquecer a análise, podemos sobrepor uma **curva de distribuição Normal** teórica no gráfico. Isso nos permite ver visualmente o quão bem (ou mal) os retornos do nosso ativo se encaixam em uma distribuição Normal. Como vimos na estatística de Curtose, os retornos de ativos geralmente têm "caudas mais pesadas", e o histograma deixa isso claro.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm


# Usando a série de retornos do IBOV que já calculamos e limpamos
retornos_ibov = ibov['retorno_log'].dropna()

# --- Plotando o Histograma com a Curva de Densidade dos Dados (KDE) ---
plt.figure(figsize=(12, 6))
sns.histplot(retornos_ibov, bins=100, kde=True, stat='density', label='Histograma dos Retornos')

# --- Sobrepondo a Curva de Distribuição Normal Teórica ---
# Calculamos a média e o desvio padrão dos nossos dados
mu, std = retornos_ibov.mean(), retornos_ibov.std()

# Geramos uma série de pontos para a curva x
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)

# Calculamos a densidade de probabilidade (PDF) da normal para esses pontos
p = norm.pdf(x, mu, std)

# Plotamos a curva normal teórica em vermelho
plt.plot(x, p, linewidth=2, color='red', label='Curva Normal Teórica')

plt.title('Distribuição dos Retornos do IBOV vs. Distribuição Normal')
plt.legend()
plt.show()

### **5.3.1 Medida de Risco: Value at Risk (VaR)**

O **Value at Risk (VaR)** é uma das medidas de risco mais famosas do mercado financeiro. Ele busca responder à pergunta:

> "Qual é a **perda máxima esperada** (em % ou em R$) em um determinado período, com um determinado nível de confiança?"

Por exemplo, um "VaR de 95% para 1 dia" de R$1.000,00 significa que temos 95% de confiança de que nossa perda em um dia não será maior que R$1.000,00. Ou, visto de outra forma, há uma probabilidade de 5% de que a perda seja *pior* que R$1.000,00.

O método mais simples para calcular o VaR é o **paramétrico**, que assume que os retornos seguem uma distribuição Normal.

In [ ]:
from scipy.stats import norm

# Parâmetros para o cálculo
nivel_confianca = 0.95
investimento = 1_000_000  # Investimento de R$1.000.000,00

# Usando a média (mu) e desvio padrão (std) dos retornos diários que já calculamos
mu = retornos_ibov.mean()
std = retornos_ibov.std()

# Calculando o VaR
# norm.ppf encontra o ponto na curva normal correspondente à probabilidade acumulada.
# Usamos (1 - nivel_confianca) para pegar a cauda esquerda (perdas).
var_retorno = norm.ppf(1 - nivel_confianca, loc=mu, scale=std)

# Calculando o valor financeiro da perda
var_financeiro = investimento * var_retorno

print(f"--- Cálculo do VaR Paramétrico (Diário) ---")
print(f"Retorno no pior caso ({100-nivel_confianca*100:.0f}% de chance): {var_retorno:.4%}")
print(f"VaR ({nivel_confianca:.0%}) para um investimento de R${investimento:,.2f}: R${abs(var_financeiro):,.2f}")
print(f"\nIsso significa que temos {nivel_confianca:.0%} de confiança de que a perda em um dia não será maior que R${abs(var_financeiro):,.2f}.")

### **5.3.2 Medida de Risco: Drawdown**

Enquanto o VaR é uma estimativa estatística de perdas futuras, o **Drawdown** é uma medida histórica. Ele mede a **queda percentual de um ativo desde o seu último pico (topo) até o seu ponto mais baixo (fundo)**, antes que um novo pico seja atingido.

O Drawdown nos ajuda a responder perguntas como:
* "Qual foi a pior sequência de perdas que meu investimento sofreu no passado?" 
* "Quanto tempo o ativo demorou para se recuperar de uma grande queda?" 

Para calculá-lo em Python, seguimos 3 passos:
1.  Calculamos o retorno acumulado (o "índice de riqueza").
2.  Calculamos o valor máximo acumulado até cada data (os "picos anteriores").
3.  Calculamos a diferença percentual entre o valor atual e o pico anterior.

In [ ]:
# Usando o DataFrame 'ibov' que já temos
# Garanta que a célula onde 'ibov' é definido foi executada

# Passo 1: Calcular o Índice de Riqueza (retorno acumulado)
# Usamos a soma acumulada (.cumsum()) dos log returns e a exponencial (np.exp)
ibov['indice_riqueza'] = np.exp(ibov['retorno_log'].cumsum())

# Passo 2: Calcular os picos anteriores usando .cummax() (máximo acumulado)
ibov['pico_anterior'] = ibov['indice_riqueza'].cummax()

# Passo 3: Calcular o Drawdown
ibov['drawdown'] = (ibov['indice_riqueza'] - ibov['pico_anterior']) / ibov['pico_anterior']

# --- Visualizando o Gráfico de Drawdown ---
plt.figure(figsize=(12, 6))
plt.plot(ibov.index, ibov['drawdown'])
plt.fill_between(ibov.index, ibov['drawdown'], 0, color='red', alpha=0.3)
plt.title('Drawdown do IBOVESPA (2014-2023)')
plt.ylabel('Queda Percentual desde o Pico')
plt.grid(True)
plt.show()

# --- Encontrando a Pior Queda (Drawdown Máximo) ---
pior_drawdown = ibov['drawdown'].min()
data_pior_drawdown = ibov['drawdown'].idxmin() # Encontra a data do pior drawdown

print(f"O pior drawdown no período foi de {pior_drawdown:.2%}.")
print(f"Ele ocorreu na data: {data_pior_drawdown.strftime('%d/%m/%Y')}.")

### **5.3.3 Medida de Risco: Shortfall Risk (Risco de Queda)**

Enquanto o VaR foca no *valor* da perda em um certo nível de confiança, o **Shortfall Risk** (ou Risco de Queda) inverte a pergunta:

> "Qual é a **probabilidade** de o retorno do meu ativo ser pior (menor) que um determinado valor em um certo período?"

Por exemplo, podemos querer saber: "Qual a chance histórica de o IBOVESPA cair mais de 3% em um único dia?". O cálculo é empírico e muito direto: contamos quantas vezes isso aconteceu no nosso conjunto de dados e dividimos pelo número total de dias.

In [ ]:
# Usando a série de retornos do IBOV que já calculamos
retornos_ibov = ibov['retorno_log'].dropna()

# --- Cálculo do Shortfall Risk ---

# Passo 1: Definir o nosso limite de perda (threshold)
limite_de_perda = -0.03 # -3%

# Passo 2: Contar quantos dias o retorno foi MENOR que o nosso limite
# A expressão (retornos_ibov < limite_de_perda) cria uma série de True/False.
# .sum() em uma série booleana conta o número de 'True'.
dias_de_queda_extrema = (retornos_ibov < limite_de_perda).sum()

# Passo 3: Contar o número total de dias no nosso histórico
total_de_dias = retornos_ibov.count()

# Passo 4: Calcular a probabilidade (casos favoráveis / casos totais)
probabilidade = dias_de_queda_extrema / total_de_dias

print(f"--- Shortfall Risk para o IBOVESPA (2014-2023) ---")
print(f"Período analisado: {total_de_dias} dias.")
print(f"Dias em que o retorno foi pior que {limite_de_perda:.2%}: {dias_de_queda_extrema} dias.")
print(f"\nA probabilidade histórica de uma queda maior que {abs(limite_de_perda):.0%} em um dia foi de: {probabilidade:.2%}")

## **5.4 Análise em Janela Móvel (Rolling Analysis)**

Até agora, calculamos métricas como média e desvio padrão para o período histórico completo. No entanto, o comportamento de um ativo muda com o tempo. A **Análise em Janela Móvel** (ou *Rolling Analysis*) é uma técnica poderosa para observar a evolução dessas métricas.

A ideia é calcular a estatística (por exemplo, a média ou a volatilidade) não sobre todos os dados, mas sobre uma "janela" de um tamanho fixo que desliza ao longo do tempo (ex: os últimos 21 dias úteis, que representam um mês). Isso nos permite responder a perguntas como: "A volatilidade do ativo aumentou nos últimos meses?".

No Pandas, fazemos isso com o método `.rolling()`.

In [ ]:
# Usando o DataFrame 'ibov' que já temos com os retornos calculados
# Garanta que a célula onde 'ibov' é definido foi executada

# Definindo o tamanho da janela (21 dias = 1 mês de pregão)
tamanho_janela = 21

# --- Calculando a Média Móvel dos Retornos ---
ibov['media_movel_retorno'] = ibov['retorno_log'].rolling(window=tamanho_janela).mean()

# --- Visualizando a Média Móvel ---
plt.figure(figsize=(12, 6))
ibov['media_movel_retorno'].plot(label=f'Média Móvel de {tamanho_janela} dias')

# Adicionamos uma linha com a média do período todo para comparação
plt.axhline(ibov['retorno_log'].mean(), color='red', linestyle='--', label='Média do Período Completo')

plt.title('Média Móvel dos Retornos do IBOVESPA')
plt.ylabel('Retorno Log Médio')
plt.legend()
plt.grid(True)
plt.show()

A mesma análise pode ser feita com o desvio padrão para visualizar como a volatilidade (risco) do ativo evoluiu. Isso é extremamente útil para identificar períodos de alta ou baixa turbulência no mercado. 

In [ ]:
# --- Calculando a Volatilidade Móvel (Desvio Padrão Móvel) ---
ibov['vol_movel'] = ibov['retorno_log'].rolling(window=tamanho_janela).std()

# --- Visualizando a Volatilidade Móvel ---
plt.figure(figsize=(12, 6))
ibov['vol_movel'].plot(label=f'Volatilidade Móvel de {tamanho_janela} dias')

# Adicionamos uma linha com a volatilidade do período todo para comparação
plt.axhline(ibov['retorno_log'].std(), color='red', linestyle='--', label='Volatilidade do Período Completo')

plt.title('Volatilidade Móvel do IBOVESPA')
plt.ylabel('Desvio Padrão dos Retornos')
plt.legend()
plt.grid(True)
plt.show()

## **5.5 Túnel de Volatilidade (Volatility Envelope)**

O **Túnel de Volatilidade** é uma forma poderosa de visualizar os retornos diários em conjunto com a sua volatilidade móvel. O gráfico é composto por:

* Uma linha central com os **retornos diários**.
* Duas bandas, uma superior e uma inferior, que representam **múltiplos do desvio padrão móvel** (geralmente 2 desvios padrão para cima e 2 para baixo).

Essa visualização nos ajuda a identificar rapidamente:
1.  **Períodos de alta ou baixa volatilidade:** O túnel se alarga em momentos de maior risco e se estreita em momentos de calmaria.
2.  **Eventos Extremos:** Dias em que o retorno "escapa" do túnel são dias de variação atípica em relação à volatilidade recente.

In [ ]:
# Usando o DataFrame 'ibov' que já tem as colunas 'retorno_log' e 'vol_movel'

# --- Calculando as Bandas do Túnel ---
# A banda superior é 2 vezes o desvio padrão móvel
banda_superior = ibov['vol_movel'] * 2
# A banda inferior é -2 vezes o desvio padrão móvel
banda_inferior = ibov['vol_movel'] * -2

# --- Plotando o Gráfico ---
plt.figure(figsize=(15, 7))

# Plotando os retornos diários (em preto e mais finos)
plt.plot(ibov.index, ibov['retorno_log'], color='black', linewidth=0.5, label='Retorno Diário')

# Plotando as bandas de volatilidade (em vermelho)
plt.plot(ibov.index, banda_superior, color='red', linewidth=1, label='Banda de Volatilidade (+/- 2 desvios)')
plt.plot(ibov.index, banda_inferior, color='red', linewidth=1)

plt.title('Túnel de Volatilidade do IBOVESPA')
plt.ylabel('Retorno Logarítmico')
plt.xlabel('Data')
plt.legend()
plt.grid(True)
plt.show()

## **5.6 Comparando Múltiplos Ativos**

Analisar um ativo isoladamente é útil, mas o verdadeiro poder da análise de dados em finanças vem da capacidade de **comparar** diferentes ativos. Isso nos ajuda a entender quais investimentos ofereceram um melhor perfil de risco-retorno no passado. A ferramenta visual mais importante para isso é o **Gráfico de Risco vs. Retorno**.

### **5.6.1 Gráfico de Risco vs. Retorno**

Este gráfico nos permite visualizar o perfil de vários ativos em um único lugar, plotando a Volatilidade Anualizada (Risco) no eixo X e o Retorno Anualizado no eixo Y. O processo envolve 4 passos:
1. Obter os dados de preços de todos os ativos.
2. Calcular os retornos (para esta análise, usaremos retornos mensais).
3. Calcular o retorno e a volatilidade anualizados para cada ativo.
4. Construir o gráfico.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Passo 1: Obter os Dados ---
tickers = ['BOVA11.SA', 'SMAL11.SA', 'SPXI11.SA', 'PETR4.SA', 
           'VALE3.SA', 'B3SA3.SA', 'ABEV3.SA', 'ITUB4.SA']
start_date = '2016-01-01'
end_date = '2019-12-31'

print("Baixando dados...")
# Primeiro, baixamos todos os dados
dados_completos = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)

# AGORA O PASSO CRUCIAL: Selecionamos APENAS a coluna 'Close'
precos = dados_completos['Close'].dropna()


# --- Passo 2: Calcular os Retornos Mensais ---
print("Calculando retornos mensais...")
precos_mensais = precos.resample('ME').last()
retornos_mensais = np.log(precos_mensais).diff().dropna()


# --- Passo 3: Calcular Risco e Retorno Anualizados ---
print("Calculando métricas anualizadas...")
retorno_anualizado = retornos_mensais.mean() * 12
vol_anualizada = retornos_mensais.std() * np.sqrt(12)


# --- Passo 4: Construindo o Gráfico de Risco vs. Retorno ---
print("Construindo o gráfico...")
plt.figure(figsize=(12, 8))
sns.scatterplot(x=vol_anualizada, y=retorno_anualizado, s=100)

for i in range(len(retorno_anualizado)):
    plt.text(x=vol_anualizada.iloc[i], 
             y=retorno_anualizado.iloc[i], 
             s=retorno_anualizado.index[i],
             fontdict=dict(color='black', size=10),
             bbox=dict(facecolor='white', alpha=0.5))

plt.title('Gráfico de Risco vs. Retorno (2016-2019)', fontsize=16)
plt.xlabel('Volatilidade Anualizada (Risco)', fontsize=12)
plt.ylabel('Retorno Anualizado', fontsize=12)
plt.grid(True)
plt.show()

### **5.6.2 Comparando Distribuições com Boxplot**

O gráfico de Risco vs. Retorno resume cada ativo a apenas dois números (média e desvio padrão). Um **Boxplot** nos permite ver a distribuição completa dos retornos mensais de cada ativo, lado a lado.

Com ele, podemos comparar facilmente a mediana, os quartis e a dispersão dos retornos, além de identificar meses com retornos atípicos (outliers).

In [ ]:
# Usando o DataFrame 'retornos_mensais' que já calculamos

plt.figure(figsize=(15, 8))
sns.boxplot(data=retornos_mensais)

plt.title('Distribuição dos Retornos Mensais por Ativo (2016-2019)', fontsize=16)
plt.ylabel('Retorno Logarítmico Mensal', fontsize=12)
plt.xlabel('Ativos', fontsize=12)
# Adiciona uma linha horizontal em zero para referência
plt.axhline(0, color='black', linestyle='--')
plt.show()

### **5.6.3 Alternativa Visual: Violin Plot**

O **Violin Plot** é uma combinação de um Boxplot com um gráfico de densidade. Ele nos dá a mesma informação de um boxplot, mas também mostra o "formato" da distribuição dos retornos, indicando onde os retornos foram mais concentrados.

In [ ]:
plt.figure(figsize=(15, 8))
sns.violinplot(data=retornos_mensais)

plt.title('Distribuição dos Retornos Mensais por Ativo (2016-2019)', fontsize=16)
plt.ylabel('Retorno Logarítmico Mensal', fontsize=12)
plt.xlabel('Ativos', fontsize=12)
plt.axhline(0, color='black', linestyle='--')
plt.show()

## **5.6.4 Correlação entre os Ativos**

A **correlação** é uma medida estatística que indica o grau de relação linear entre o movimento de dois ativos. Ela varia de -1 a +1:
* **+1 (Correlação Perfeita Positiva):** Os ativos tendem a se mover na mesma direção.
* **0 (Sem Correlação):** O movimento de um ativo não tem relação linear com o outro.
* **-1 (Correlação Perfeita Negativa):** Os ativos tendem a se mover em direções opostas.

Entender a correlação é fundamental em finanças, pois é a base da **diversificação de carteiras**. Combinar ativos com baixa ou negativa correlação pode ajudar a reduzir o risco geral de um portfólio.

No Pandas, calculamos a matriz de correlação de todos os ativos de um DataFrame de forma muito simples com o método `.corr()`.

In [ ]:
# Usando o DataFrame 'retornos_mensais' que já calculamos

# O método .corr() calcula a correlação par a par entre todas as colunas
matriz_correlacao = retornos_mensais.corr()

print("--- Matriz de Correlação dos Retornos Mensais ---")
print(matriz_correlacao)

Uma matriz de números pode ser difícil de interpretar rapidamente. Uma forma muito mais eficaz de visualizar uma matriz de correlação é usando um **Heatmap (Mapa de Calor)**. Nele, cores mais quentes (vermelho) indicam alta correlação positiva, e cores mais frias (azul) indicam correlação negativa.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(matriz_correlacao, 
            annot=True,        # Mostra os números dentro de cada célula
            cmap='coolwarm',   # Define o esquema de cores (vermelho-branco-azul)
            fmt='.2f',         # Formata os números para 2 casas decimais
            linewidths=.5)

plt.title('Heatmap de Correlação dos Ativos (2016-2019)', fontsize=16)
plt.show()

## Fim do Capítulo 5

Parabéns! Você concluiu um capítulo denso e muito importante. Agora você tem um kit de ferramentas robusto para análise de risco. Você aprendeu a:

* Analisar e visualizar o comportamento de séries de retornos.
* Calcular estatísticas descritivas avançadas, como assimetria e curtose.
* Quantificar o risco através de métricas como **VaR** e **Drawdown**.
* Analisar a evolução do risco e retorno com **janelas móveis**.
* Comparar múltiplos ativos usando gráficos de **Risco vs. Retorno**, **Boxplots** e **Heatmaps de Correlação**.

Com essa base, estamos prontos para o último capítulo do nosso curso, o **Capítulo 6: Análise de Carteiras**, onde vamos usar a correlação e a volatilidade para começar a construir portfólios otimizados.